# Math Is Fun!!!!!!
Linear algebra and calculus are used to blah blah the blah blah so you can blah blah machine learning.


![now for something completely different](completely-different.jpg)


# Why oh why would do I do this?
The problems machine learning solves are real pain in the neck problems. When you have some problem that looks like it is going to take one billion if-then-else statements, use machine learning to solve it.

__A Definition__

Artificial intelligence: a computer program used to solve an intractable problem.

__An Example__

The "hello world" of Kaggle is the morose task of figuring out who dies in the fridgid waters of the north atlantic when the Titanic tragically collides with an iceberg and sinks to the murky depths.

SPOILER ALERT! Kate Winslet survives but Leonardo DiCaprio is turned into a dead human popcicle and sinks beneath the whitecaps. Can we generalize this idea? It seemed like women and children were piled onto the lifeboats while men and chello players did not fare as well. Is it cynical to guess that rich people survive and steerage class die? Let's use MACHINE LEARNING to test the hypothesis that sex, age, and ticket price determines if a passenger survives the Titanic disaster.


# BUT FIRST, DATA SCIENCE IS MUNGING!

In [ ]:
# Let's load the data and take a look...
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
data = pd.read_csv('../input/train.csv')
print(data[:5]) # print out first 5 rows

In [ ]:
# Hmmm, let's focus on Sex, Age, and what they paid for their ticket
print(data[['Survived', 'Sex', 'Age', 'Fare']][:10])

There are three problems with this data:

1. It all must be numeric data for our ML maths to crunch it.
2. That NaN there is going to cause all kinds of trouble.
3. (Just trust me) all the data needs to be on the same scale.

The solutions for these problems:

1. Convert male/female to 0 or 1
2. Make some default for missing NaN data
3. "<a href="https://en.wikipedia.org/wiki/Normalization_(statistics)">Normalize</a>" the data. We will use feature scaling in this case to jam everything in to the range [0-1].

In [ ]:
# No snickering
def sex_conversion(row):
    if row['Sex'] == 'female':
        return 1
    else:
        return 0

data1 = data.copy()
data1['norm_sex'] = data1.apply(sex_conversion, axis=1)

print(data1[['Survived', 'Sex', 'norm_sex', 'Age', 'Fare']][:10])

In [ ]:
# Fill in missing data with the median. Other strategies include:
#  Remove the row or use the most frequently occuring value.
#  Interestingly, we could create another ML model to predict a row's missing
#  values and fill in with that!
data2 = data1.fillna(data.median())
print(data2[['Survived', 'norm_sex', 'Age', 'Fare']][:10])

In [ ]:
def feature_scaler(df, col):
    df['norm_' + col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    
data3 = data2.copy()
feature_scaler(data3, 'Age')
feature_scaler(data3, 'Fare')
print(data3[['Survived', 'norm_sex', 'Age', 'norm_Age', 'Fare', 'norm_Fare']][:10])

__Great!__ Our data is now all munged into submission.

The next step is to extract our survivor flag into an array (vector). This is the "label" for our supervised machine learning problem.

Then, we need to split our input features and labels into two groups: one for training and one for testing.

There are nice utilities for these tasks. Also, we will start calling our data X and y since they are our dependent and indepednet variables.

In [ ]:
import numpy as np # linear algebra
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

y = np.array(data3['Survived'].values)
X = data3[['norm_sex', 'norm_Age', 'norm_Fare']].values # we have three features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
print(y[:10])
print(X[:10])

# ...and now presenting Keras
Keras is a neural network API that abstracts low-level toolkits like Theno and TensorFlow. It makes it really easy to read the code and abstracts away a lot of boilerplate. If you think of a neural network as something like this:

![Nerual Network](Colored_neural_network.svg)

Using Keras will take your intuition and turn it into code very easily.

The basic idea is that we define a neural netowrk architecture, compile it down to the underlying implementation, and then fit the model with our training data. Once that is done, we can use the model to predict new observations.

Let's code it.


In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(20, input_dim=3, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, batch_size=50, epochs=20, verbose=0)

In [ ]:
prediction = model.predict(X_test, verbose=0)

correct = 0
for i in range(0, prediction.shape[0]):
    if np.argmax(prediction[i]) == np.argmax(y_test[i]):
        correct = correct + 1

accuracy = correct / prediction.shape[0]
print("\nDONE. %f" % accuracy)

Does that say "DONE. 1.000000" up there? That means 💯!

WOW! Magic!

Let's put that in terms of our original hypothesis. We can say that given sex, age, and ticket price our model can determine Titanic disaster survival.